In [ ]:
import json
import re
from collections import defaultdict

In [ ]:
dirs = !ls -d ./slackdata/*/

In [ ]:
users = dict()
with open('./slackdata/users.json') as f:
    j = json.loads(f.read())
    for user in j:
        userid = ''
        profile = user['profile']
        if 'real_name_normalized' in profile.keys(): userid = profile['real_name_normalized']
        if 'email' in profile.keys(): userid = profile['email']
        users[user['id']] = userid.replace('.','_').replace(' ', '_').lower()

In [ ]:
json_files = []
for d in dirs:
    files = !ls {d}*.json
    for f in files:
        json_files.append(f)

In [ ]:
s = ''
for jf in json_files:
    with open(jf) as f:
        j = json.loads(f.read())
        for item in j:
            if item['type'] == 'message':
                user = 'unknown_user'
                text = 'text'
                try:
                    user = item['user']
                except:
                    try:
                        user = item['comment']['user']
                    except:
                        user = 'unknown_user'
                try:
                    user = users[user]
                except:
                    user = 'unknown_user'
                try:
                    text = item['text']
                except:
                    text = ''
                s = '{} <__{}__> {}'.format(s, user, text)

In [ ]:
def repl(match):
    try:
        return '@' + users[match.group(0)[2:-1]]
    except:
        return '@unknown_user '
news = re.sub("<@(W[0-9A-Z]*)>", repl, s)
data = re.sub(r'<http[s]?://(\S+.*)>', '<FILE_OR_URL>', news)

In [ ]:
sents = data.replace('\t','')
sents = sents.replace('\r','')
sents = sents.split('\n')

In [ ]:
sents = [sent for sent in sents if len(sent) > 1]

In [ ]:
with open('./slack.parsed', 'w') as outfile:
    outfile.write(' '.join(sents))